## function to get shorter excerpt of samples
### criterion: highest stdev in `range_` samples

In [235]:
def get_excerpt(arr, range_=10, grain=10, get='excerpt', include_idx=False):
    # assume arr is 1-D ndarray
    stds = []

    # approximate with rough grain for faster execution
    # restrict right end for full excerpt
    for i in range(0, len(arr)-range_, grain):
        stds.append(arr[i:i+range_].std())

    stds = np.array(stds)
    max_idx = np.argmax(stds)*grain
    #print(stds, stds.max(), max_idx)
    #print(str(max_idx)[:-2], end=' ')
    
    excerpt = arr[max_idx:max_idx+range_]
    
    if include_idx:
        excerpt = np.concatenate([np.array([max_idx]), excerpt], axis=0)
    
    
    return eval(get)

## get data and truncate

In [179]:
%%time
df_data = pd.read_csv('../../projects/proj3/_data/combined_csv/180116_87300samp_cry386_laugh703_with_ids_mu_law.csv')

CPU times: user 1min 49s, sys: 1.06 s, total: 1min 50s
Wall time: 1min 50s


In [249]:
%%time

new_data = []

for idx, row in df_data.iterrows():
    new_data.append(get_excerpt(np.array(row[1:-1]), range_=3600, grain=500, include_idx=True))

CPU times: user 5.72 s, sys: 220 ms, total: 5.94 s
Wall time: 5.94 s


In [256]:
new_data = np.array(new_data)
new_data_idces = new_data[:, 0]
new_data = new_data[:, 1:]

df_new = pd.concat([df_data['sound_id'],
           pd.Series(new_data_idces, dtype=np.int32, name='start_idx'),
           pd.DataFrame(new_data),
           df_data['laugh']],
          axis=1)

### save new df

In [263]:
%%time
df_new.to_csv('../../projects/proj3/_data/combined_csv/180126_3600samp_cry386_laugh703_with_ids_mu_law.csv',
             index=False)

CPU times: user 2.27 s, sys: 89.5 ms, total: 2.36 s
Wall time: 2.48 s


## check saved file

In [264]:
%%time
df_new = pd.read_csv('../../projects/proj3/_data/combined_csv/180126_3600samp_cry386_laugh703_with_ids_mu_law.csv')

CPU times: user 596 ms, sys: 15.9 ms, total: 612 ms
Wall time: 622 ms


In [265]:
df_new.sample(3)

,sound_id,start_idx,0,1,2,3,4,5,6,7,...,3591,3592,3593,3594,3595,3596,3597,3598,3599,laugh
98,80375,20500,33.0,34.0,39.0,49.0,66.0,124.0,190.0,202.0,...,204.0,190.0,104.0,58.0,44.0,37.0,35.0,35.0,38.0,0.0
302,263402,68000,70.0,68.0,70.0,68.0,64.0,63.0,60.0,55.0,...,203.0,203.0,203.0,206.0,208.0,205.0,202.0,205.0,208.0,0.0
606,20688,19000,60.0,89.0,148.0,119.0,67.0,51.0,50.0,52.0,...,48.0,71.0,194.0,205.0,184.0,60.0,51.0,69.0,192.0,1.0


In [266]:
df_new.shape

(1089, 3603)